Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (18724, 28, 28) (18724,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (18724, 784) (18724, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### 1-Layer perceptron with L2 regulatisation

In [14]:
batch_size = 128
l2_coef_np = np.float32(0.001)

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  l2_coef = tf.constant(l2_coef_np)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + l2_coef*(tf.nn.l2_loss(weights))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [15]:
num_steps = 10001


with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 3000 == 0):
      print("Minibatch loss at step", step, ":", l)
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0 : 21.1469
Minibatch accuracy: 11.7%
Validation accuracy: 15.8%
Minibatch loss at step 3000 : 0.819487
Minibatch accuracy: 81.2%
Validation accuracy: 81.8%
Minibatch loss at step 6000 : 0.658115
Minibatch accuracy: 82.8%
Validation accuracy: 81.6%
Minibatch loss at step 9000 : 0.482467
Minibatch accuracy: 90.6%
Validation accuracy: 81.2%
Test accuracy: 88.8%


### Problem 1 for 1-Layer perceptron results

For 3001 steps, 1-layer, SGD step 0.5:  
 - l2_coef=0, test_acc = 86.0%  
 - l2_coef=0.1, test_acc = 84.1%  
 - l2_coef=0.2, test_acc = 81.6%
 - l2_coef=0.3, test_acc = 78.4%
 
For 10001 steps, 1-layer, SGD step 0.5:
 - l2_coef=0, test_acc = 88.0%
 - l2_coef=0.0001, test_acc = 88.6%
 - l2_coef=0.001 test_acc = 88.8%
 - l2_coef=0.01, test_acc = 88.4%
 - l2_coef=0.1, test_acc = 85.8%
 - l2_coef=0.2, test_acc = 82.0%
 - l2_coef=0.3, test_acc = 76.6%

## Problem 1 for One hidden layer nn

In [17]:
batch_size = 128
l2_coef_np = np.float32(0.001)

num_hid = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  l2_coef = tf.constant(l2_coef_np)
  
  # Variables.
  #hidden layer
  w_hid = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hid]))
  b_hid = tf.Variable(tf.zeros([num_hid]))
  #output layer  
  weights = tf.Variable(
    tf.truncated_normal([num_hid, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  out_hid = tf.nn.relu(tf.matmul(tf_train_dataset, w_hid) + b_hid)
  logits = tf.matmul(out_hid, weights) + biases
  loss = (tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    + l2_coef*tf.nn.l2_loss(weights) + l2_coef*tf.nn.l2_loss(w_hid) )
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss) 
                
    
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  v_out_hid = tf.nn.relu(tf.matmul(tf_valid_dataset, w_hid) + b_hid)
  v_logits = tf.matmul(v_out_hid, weights) + biases
  valid_prediction = tf.nn.softmax(v_logits)
  t_out_hid = tf.nn.relu(tf.matmul(tf_test_dataset, w_hid) + b_hid)
  t_logits = tf.matmul(t_out_hid, weights) + biases  
  test_prediction = tf.nn.softmax(t_logits)

In [6]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 5000 == 0):
      print("Minibatch loss at step", step, ":", l)
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0 : 325.629
Minibatch accuracy: 3.9%
Validation accuracy: 36.5%
Minibatch loss at step 5000 : 2.36266
Minibatch accuracy: 84.4%
Validation accuracy: 82.4%
Minibatch loss at step 10000 : 0.513931
Minibatch accuracy: 93.0%
Validation accuracy: 84.2%
Test accuracy: 90.6%


### Problem 1 for One hidden layer results
For 10001 steps, SGD step 0.5:
 - l2_coef=0, test_acc = 90.6%
 - l2_coef=0.001, test_acc = 94.5%

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [18]:
#overfitting
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    #offset = (step * batch_size) % (5*batch_size)
    offset = 128 #one batch
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 2000 == 0):
      print("Current offset: ", offset)
      print("Minibatch loss at step", step, ":", l)
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Current offset:  128
Minibatch loss at step 0 : 680.682
Minibatch accuracy: 2.3%
Validation accuracy: 26.8%
Current offset:  128
Minibatch loss at step 2000 : 42.6959
Minibatch accuracy: 100.0%
Validation accuracy: 62.8%
Current offset:  128
Minibatch loss at step 4000 : 5.77713
Minibatch accuracy: 100.0%
Validation accuracy: 64.5%
Current offset:  128
Minibatch loss at step 6000 : 0.798418
Minibatch accuracy: 100.0%
Validation accuracy: 68.9%
Current offset:  128
Minibatch loss at step 8000 : 0.130796
Minibatch accuracy: 100.0%
Validation accuracy: 69.5%
Current offset:  128
Minibatch loss at step 10000 : 0.0395091
Minibatch accuracy: 100.0%
Validation accuracy: 70.0%
Test accuracy: 76.6%


### Problem 2 for One-hidden layer nn:
SGD step 0.5, L2 regulatisation coef = 0.001, 10001 
... maybe i don't understand something? is this model too small to overfit?
 - 1 batch,   test_acc=76.6%, not too bad, eh?
 - 5 batches, test_acc=85.9%
 - 10 batches, test_acc=87.4%

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [15]:
batch_size = 128
l2_coef_np = np.float32(0.001)
keep_prop_np = np.float32(0.7)

num_hid = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  l2_coef = tf.constant(l2_coef_np)
  keep_prop = tf.constant(keep_prop_np)
  
  # Variables.
  #hidden layer
  w_hid = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hid]))
  b_hid = tf.Variable(tf.zeros([num_hid]))
  #output layer  
  weights = tf.Variable(
    tf.truncated_normal([num_hid, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  out_hid = tf.nn.relu(tf.matmul(tf_train_dataset, tf.nn.dropout(w_hid,keep_prop)) + b_hid)
  logits = tf.matmul(out_hid, tf.nn.dropout(weights,keep_prop)) + biases
  loss = (tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    + l2_coef*tf.nn.l2_loss(weights) + l2_coef*tf.nn.l2_loss(w_hid) )
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss) 
                
    
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  v_out_hid = tf.nn.relu(tf.matmul(tf_valid_dataset, w_hid) + b_hid)
  v_logits = tf.matmul(v_out_hid, weights) + biases
  valid_prediction = tf.nn.softmax(v_logits)
  t_out_hid = tf.nn.relu(tf.matmul(tf_test_dataset, w_hid) + b_hid)
  t_logits = tf.matmul(t_out_hid, weights) + biases  
  test_prediction = tf.nn.softmax(t_logits)

In [16]:
#overfitting
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (5*batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 2000 == 0):
      print("Current offset: ", offset)
      print("Minibatch loss at step", step, ":", l)
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Current offset:  0
Minibatch loss at step 0 : 748.344
Minibatch accuracy: 17.2%
Validation accuracy: 33.5%
Current offset:  0
Minibatch loss at step 2000 : 44.7265
Minibatch accuracy: 100.0%
Validation accuracy: 77.4%
Current offset:  0
Minibatch loss at step 4000 : 6.05741
Minibatch accuracy: 100.0%
Validation accuracy: 78.1%
Current offset:  0
Minibatch loss at step 6000 : 0.862786
Minibatch accuracy: 100.0%
Validation accuracy: 78.9%
Current offset:  0
Minibatch loss at step 8000 : 0.174863
Minibatch accuracy: 100.0%
Validation accuracy: 78.5%
Current offset:  0
Minibatch loss at step 10000 : 0.0832079
Minibatch accuracy: 100.0%
Validation accuracy: 78.7%
Test accuracy: 86.0%


### Problem 3 for One-hidden layer nn results:
SGD step 0.5, L2 regulatisation coef = 0.001, 10001 steps 
added dropout, and ...
 - 5 batches, test_acc=86.0%(+0.1% after adding dropout)
 - 10 batches, test_acc=87.4%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [22]:
batch_size = 128
l2_coef_np = np.float32(0.001)
keep_prop_np = np.float32(0.5)

num_hid = 1024
num_steps = 10001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  l2_coef = tf.constant(l2_coef_np)
  keep_prop = tf.constant(keep_prop_np)
  
  # Variables.
  #hidden layer
  w_hid = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hid]))
  b_hid = tf.Variable(tf.zeros([num_hid]))
  #output layer  
  weights = tf.Variable(
    tf.truncated_normal([num_hid, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  global_step = tf.Variable(0, trainable=False)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, num_steps,0.96,staircase=True)
  
  # Training computation.
  out_hid = tf.nn.relu(tf.matmul(tf_train_dataset, tf.nn.dropout(w_hid,keep_prop)) + b_hid)
  logits = tf.matmul(out_hid, tf.nn.dropout(weights,keep_prop)) + biases
  loss = (tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    + l2_coef*tf.nn.l2_loss(weights) + l2_coef*tf.nn.l2_loss(w_hid) )
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step) 
                
    
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  v_out_hid = tf.nn.relu(tf.matmul(tf_valid_dataset, w_hid) + b_hid)
  v_logits = tf.matmul(v_out_hid, weights) + biases
  valid_prediction = tf.nn.softmax(v_logits)
  t_out_hid = tf.nn.relu(tf.matmul(tf_test_dataset, w_hid) + b_hid)
  t_logits = tf.matmul(t_out_hid, weights) + biases  
  test_prediction = tf.nn.softmax(t_logits)

In [23]:


with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 2000 == 0):
      print("Minibatch loss at step", step, ":", l)
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0 : 892.508
Minibatch accuracy: 13.3%
Validation accuracy: 19.0%
Minibatch loss at step 2000 : 129.862
Minibatch accuracy: 64.8%
Validation accuracy: 71.6%
Minibatch loss at step 4000 : 15.8795
Minibatch accuracy: 69.5%
Validation accuracy: 79.9%
Minibatch loss at step 6000 : 3.05769
Minibatch accuracy: 71.9%
Validation accuracy: 82.1%
Minibatch loss at step 8000 : 1.13473
Minibatch accuracy: 80.5%
Validation accuracy: 83.3%
Minibatch loss at step 10000 : 0.753425
Minibatch accuracy: 78.1%
Validation accuracy: 83.0%
Test accuracy: 90.1%


### Problem 4 for One-hidden layer nn results:
SGD step 0.5 with exp decay(0.96), L2 regulatisation coef = 0.001, 10001 steps, dropout=0.5, test_acc = 90.1% (
